In [9]:
from sentence_transformers import SentenceTransformer,util
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import json
import torch

data = pd.read_json("data/climate-fever.jsonl", lines=True)

for ev in data["evidences"][0]:
    print(ev["evidence"])

"Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event".
Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral reefs, mountains, and the Arctic.
Rising temperatures push bees to their physiological limits, and could cause the extinction of bee populations.
Rising global temperatures, caused by the greenhouse effect, contribute to habitat destruction, endangering various species, such as the polar bear.
"Bear hunting caught in global warming debate".


In [37]:
indexes = np.arange(len(data))
np.random.shuffle(indexes)

train_index = np.arange(int(len(data)*0.8))
test_index = np.arange(int(len(data)*0.8),len(data))

print(indexes[train_index])
print(indexes[test_index])

[ 323 1383  157 ... 1083  347  204]
[ 457  905  188  925  971  926 1401 1198 1400  439 1508   22  792   20
 1409 1507  992 1217  338 1382  691  431 1386  608  161 1518  466 1178
 1192 1022  185  140 1177   81 1231  624  658  842  781 1305  407 1299
   36  633 1173  985  749  774  585  917  729   94  646  929  869  132
  443   68  405 1310  804 1478  938  871  662 1420   84 1147  378  361
  110 1384  438 1201  250  635  837  919   43 1228 1030 1180 1463  235
  873  316  784 1206  404  768 1245 1020  186  965  945  232  739  424
  100 1522 1086  524 1432  851 1001  514   82  996  901  897 1402 1047
  227 1214  397 1341  607  319  230  979 1063  666  420 1346 1502 1216
  708  776  943 1078   61 1244  664  345  498 1248 1447  715 1407  234
    6 1100 1318  109  931  629   56 1332  941 1009 1067  497 1186  268
  571  687  422  689 1128 1292  688 1019  780 1380  379 1137 1108    7
 1140  593  297  493  713 1115  483 1397 1485  489 1527  728  686  665
  164 1152  648  757 1455  496  906  723 